In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Create an effective prompt template for getting flights")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Create an effective prompt template for getting flights', additional_kwargs={}, response_metadata={}, id='310aaf9e-6476-4715-9bbc-08bedc55bcab'),
              AIMessage(content='**Flight‑Search Prompt Template**\n\nUse the following template to request flight information from the Kiwi API.  \nReplace the placeholders (in brackets) with the user’s preferences before calling the `search_flight` function.\n\n```json\n{\n  "cabinClass": "[M|W|C|F]",          // M=Economy, W=Economy Premium, C=Business, F=First\n  "curr": "[USD|EUR|GBP|JPY|CAD|AUD|NZD|CHF]",  // Currency code\n  "departureDate": "[dd/mm/yyyy]",    // e.g., 15/07/2026\n  "departureDateFlexRange": [0-3],    // Days before/after the departure date\n  "flyFrom": "[City or Airport Code]", // e.g., "New York" or "JFK"\n  "flyTo": "[City or Airport Code]",   // e.g., "London" or "LHR"\n  "locale": "[en|de|fr|es|it|ru|uk|...]", // Language code for city names & links\n  "passengers": {\n    "adu

In [ ]:
print(response["messages"][-1].content)

In [ ]:
user_prompt = """
Can you use the json below to create an appropriate prompt:


"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config
    )

pprint(response)

In [ ]:
print(response["messages"][-1].content)

In [ ]:
user_prompt = """
Find me a one-way economy flight from Paris CDG to Barcelona BCN on July 15, 2026. 
I'm flexible by one day and want the cheapest price in EUR.
"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config
    )

pprint(response)

In [ ]:
print(response["messages"][-1].content)

In [ ]:
user_prompt = """
I'm planning a summer trip. Can you look for flights for 2 adults and 1 child 
from New York JFK to Tokyo NRT? We want to leave on August 1st, 2026, and come 
back ten days later. Business class and sort by the shortest duration.
"""

response = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config
    )

pprint(response)

In [ ]:
print(response["messages"][-1].content)